# Data cleaning practice. Audible dataset #

Importing the necessary libraries:

In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import partial

You can find the dataset in [https://www.kaggle.com/datasets/snehangsude/audible-dataset](https://www.kaggle.com/datasets/snehangsude/audible-dataset). All credit to the author.

I will be using an already downloaded version.

In [150]:
wd = os.getcwd()+"/"
audible_uncleaned = pd.read_csv(wd+"audible_uncleaned.csv")

Let's explore the dataframe:

In [151]:
audible_uncleaned.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [152]:
#name. Number of books can be extracted depending on the case
#author: remove written by and separate names
#narrator: idem
#time:convert to datetime64
#releasedate: convert to datetime 64
#language: --
#stars: separate stars and ratings
#price: convert to euros.

In [153]:
audible_uncleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


With this information, it seems evident that there are a number of things that can be done:
- name. Number of books, and the series could be extracted. Difficult, given the heterogeneous way in which it is expressed.
- author: remove "Written by" and transform CamelCase into proper names with spaces.
- narrator: same thing that for author
- time:convert from text to timedelta
- releasedate: convert to datetime 64
- language: Nothing to be done here.
- stars: separate stars and number of ratings.
- price: in the information of the dataset it is stated that prices are in INR. They can be trasnformed to floats and converted to euros.

Now, let's created an empty dataframe in which we can add the cleaned columns:

In [154]:
audible_cleaned =pd.DataFrame()

## Name column ##

For now, name column will be left as it is:

In [155]:
audible_cleaned["name"]=audible_uncleaned["name"].copy()

## Author column ##

To clean author column we need two functions:

- The first, to remove "Written by":

In [156]:
def extract_text_left_of (strReference, serToProcess):
    return serToProcess.str.extract(r""+strReference+"(.*)")

- And the second to split CamelCase words with spaces:

In [157]:
def splitCamelCase(str):
    def addAsteriskInUpperCase(str):
        new_string="*"
        for i in str:
            if(i.isupper() and new_string[-1].isalpha()): #In addtion, previous character must be a letter. To prevent split in O'Brian for instance
                new_string+="*"+i
            else:
                new_string+=i
        return new_string

    def splitStrOnAsterisk(str):
        return str.split("*")
    def removeEmpty(lst):
        return [n for n in lst if not(n=='')]
    def joinListWithSpaces(lst):
        return " ".join(lst)
    return joinListWithSpaces(removeEmpty(splitStrOnAsterisk( addAsteriskInUpperCase(str) ))) 

Now, we can apply them to the original column:

In [158]:
audible_cleaned["author"]=(
                            audible_uncleaned["author"]
                            .pipe((extract_text_left_of,"serToProcess"),strReference="Writtenby:")\
                            .pipe(np.vectorize(splitCamelCase))
                            )
audible_cleaned["author"].head()

0    Geronimo Stilton
1        Rick Riordan
2         Jeff Kinney
3        Rick Riordan
4        Rick Riordan
Name: author, dtype: object

## Narrator column ##

Same process than with author:

In [159]:
audible_cleaned["narrator"]=audible_uncleaned["narrator"]\
                            .pipe((extract_text_left_of,"serToProcess"),strReference="Narratedby:")\
                            .pipe(np.vectorize(splitCamelCase))

In [160]:
audible_cleaned["narrator"].head()

0        Bill Lobely
1     Robbie Daymond
2        Dan Russell
3    Soneela Nankani
4    Jesse Bernstein
Name: narrator, dtype: object

## Time column ##

We need to convert it to timedelta:

In [161]:

audible_cleaned["time"]= (pd.to_timedelta( 
                                        audible_uncleaned["time"]
                                            .str.replace(" and ", " ")
                                            .str.replace("hr", "hour")
                                            .str.replace("mins","min")
                                            .str.replace("Less than 1 minute","60s")
                                        )
                        )

## Release Date column ##

And this column to datetime64:

In [162]:
audible_cleaned["releasedate"]= pd.to_datetime(audible_uncleaned["releasedate"], format='%d-%m-%y')

## Stars column ##

First, we need to extract the numbers. To do that we create a regex pattern and apply it to the first occurrence to extract the stars rated:

In [163]:

regex_int_or_float="\d*\.?\d*"
audible_cleaned["stars_rated"]=(
                                audible_uncleaned["stars"]
                                    .str.extract(r"({g1})\s".format(g1=regex_int_or_float),expand=False)\
                                    .pipe(pd.to_numeric,errors='coerce')
                                )

By coercing errors in pd.to_numeric, books that have not been rated ("Not rated yet") are converted to NaN.

We can also extract the maximum rating for every row:

In [164]:
audible_cleaned["max_stars"]=(
    audible_uncleaned["stars"]
    .str.extract(r"^(?:{g1}) out of ({g1})".format(g1=regex_int_or_float),expand=False)\
    .pipe(pd.to_numeric,errors='coerce')
)

And check that all are calculated taking 5 stars as the maximum:

In [165]:
audible_cleaned["max_stars"].value_counts()

5.0    15072
Name: max_stars, dtype: int64

The numbers of ratings can be extracted using the same pattern. In this case, NaN values (for "Not rated yet") are filled with 0s.

In [166]:
audible_cleaned["ratings"]=(
     audible_uncleaned["stars"]
    .str.extract(r"^(?:{g1}) out of (?:{g1}) stars({g1}) rating(?:s?)".format(g1=regex_int_or_float),expand=False)
    .pipe(pd.to_numeric,errors='coerce')
    .fillna(0)
)

We also add a normalised rating column:

In [167]:
audible_cleaned["normalised_stars_rating"]= audible_cleaned.eval("stars_rated / max_stars" )

## Language column ##

Explorign the language column we can see that to make the column more homogenuous we can capitalise all the languages and remove special characters (as in "mandarin_chinese"):

In [168]:
audible_uncleaned["language"].value_counts()

English             61884
german               8295
spanish              3496
japanese             3167
italian              2694
french               2386
russian              1804
danish                935
portuguese            526
swedish               515
Hindi                 436
polish                224
finnish               197
dutch                 190
tamil                 161
catalan               153
mandarin_chinese       97
icelandic              52
romanian               50
hungarian              36
urdu                   34
afrikaans              28
czech                  23
turkish                20
greek                  18
arabic                 16
norwegian              16
galician               10
bulgarian               9
korean                  4
slovene                 4
hebrew                  2
basque                  2
telugu                  2
lithuanian              2
ukrainian               1
Name: language, dtype: int64

In [169]:
audible_cleaned["language"]=(
    audible_uncleaned["language"]
    .str.replace(pat=r"_", repl=r" ")
    .str.capitalize()
)

audible_cleaned["language"].value_counts()

English             61884
German               8295
Spanish              3496
Japanese             3167
Italian              2694
French               2386
Russian              1804
Danish                935
Portuguese            526
Swedish               515
Hindi                 436
Polish                224
Finnish               197
Dutch                 190
Tamil                 161
Catalan               153
Mandarin chinese       97
Icelandic              52
Romanian               50
Hungarian              36
Urdu                   34
Afrikaans              28
Czech                  23
Turkish                20
Greek                  18
Arabic                 16
Norwegian              16
Galician               10
Bulgarian               9
Korean                  4
Slovene                 4
Hebrew                  2
Basque                  2
Telugu                  2
Lithuanian              2
Ukrainian               1
Name: language, dtype: int64

## Price column ##

And now, the price column:

In [170]:
audible_uncleaned["price"][0:10]

0      468.00
1      820.00
2      410.00
3      615.00
4      820.00
5      656.00
6      233.00
7      820.00
8    1,256.00
9      820.00
Name: price, dtype: object

To avoid errors with pd.to_numeric, we need to suppress the comma separators. Also, we replace the "Free" text with a 0s:

In [171]:
audible_cleaned["price_INR"]= (
    audible_uncleaned["price"]
    .str.replace(r",","", regex=True)
    .str.replace(r"Free","0", regex=True)
    .pipe(pd.to_numeric)
)

In [172]:
audible_cleaned["price_EUR_calc"] = audible_cleaned["price_INR"]*0.012 #Converted as of 5/12/2022. To do: get exchange rate from an api

In [173]:
audible_cleaned.head()

,name,author,narrator,time,releasedate,stars_rated,max_stars,ratings,normalised_stars_rating,language,price_INR,price_EUR_calc
0,Geronimo Stilton #11 & #12,Geronimo Stilton,Bill Lobely,0 days 02:20:00,2008-08-04,5.0,5.0,34.0,1.0,English,468.0,5.616
1,The Burning Maze,Rick Riordan,Robbie Daymond,0 days 13:08:00,2018-05-01,4.5,5.0,41.0,0.9,English,820.0,9.840
2,The Deep End,Jeff Kinney,Dan Russell,0 days 02:03:00,2020-11-06,4.5,5.0,38.0,0.9,English,410.0,4.920
3,Daughter of the Deep,Rick Riordan,Soneela Nankani,0 days 11:16:00,2021-10-05,4.5,5.0,12.0,0.9,English,615.0,7.380
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,Jesse Bernstein,0 days 10:00:00,2010-01-13,4.5,5.0,181.0,0.9,English,820.0,9.840


In [174]:
audible_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   name                     87489 non-null  object         
 1   author                   87489 non-null  object         
 2   narrator                 87489 non-null  object         
 3   time                     87489 non-null  timedelta64[ns]
 4   releasedate              87489 non-null  datetime64[ns] 
 5   stars_rated              15072 non-null  float64        
 6   max_stars                15072 non-null  float64        
 7   ratings                  87489 non-null  float64        
 8   normalised_stars_rating  15072 non-null  float64        
 9   language                 87489 non-null  object         
 10  price_INR                87489 non-null  float64        
 11  price_EUR_calc           87489 non-null  float64        
dtypes: datetime64[ns](

In [175]:
audible_cleaned.to_csv("audible_cleaned.csv", index=False)